In [1]:
import hashlib 
import pandas as pd
import matplotlib.pyplot as p 

In [7]:
# загружаем файлы

In [2]:
doc_views = pd.read_csv('doc_views (1).csv')

In [4]:
import sys
print(sys.executable)  # просто посмотреть, какой именно python использует Jupyter
!{sys.executable} -m pip install openpyxl


C:\Users\Admin\AppData\Local\Programs\Python\Python314\python.exe



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
payments = pd.read_excel('case_data (1).xlsx',sheet_name='Payments')

# просмотр данных

In [17]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,Edwards,1903155229
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,Smith,1466748063
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,Parker,4864835140
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,Mcguire,1837181476
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,Erickson,5567324499


In [9]:
payments.tail(10)

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
3408,2495154,2020-11-01 12:57:31,12489750.0,0,paypal,no,NOK,AUTHORISATION,1,30
3409,2495004,2020-11-01 12:34:39,12489456.0,0,paypal,no,NOK,AUTHORISATION,1,30
3410,2494949,2020-11-01 12:18:05,12489224.0,0,paypal,no,NOK,AUTHORISATION,1,30
3411,2494822,2020-11-01 11:27:04,12488536.0,0,visa,no,NOK,AUTHORISATION,1,30
3412,2494811,2020-11-01 11:19:18,12488456.0,11997,visa,no,NOK,AUTHORISATION,0,0
3413,2494657,2020-11-01 10:43:11,8227880.0,35988,visa,no,NOK,AUTHORISATION,0,0
3414,2494297,2020-11-01 08:33:23,12487016.0,0,mc,no,NOK,AUTHORISATION,1,30
3415,2494269,2020-11-01 08:05:53,12486818.0,0,visa,no,NOK,AUTHORISATION,1,30
3416,2494216,2020-11-01 07:37:35,12486620.0,0,mc,no,NOK,AUTHORISATION,1,30
3417,2493338,2020-11-01 01:24:26,11413026.0,0,visa,no,NOK,AUTHORISATION,1,30


# Предобработка данных

## Переименование полей

In [13]:
doc_views = doc_views.rename(columns={'Name':'name','Last Name':'last_name', 'Passport Number':'passport_number', 'ID':'id'})

### Возвращаемся в ячейку doc_views.head() и заново её запускаем

## Пропуски

### Количество пропусков

In [19]:
doc_views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2241 entries, 0 to 2240
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   document_id      2241 non-null   int64         
 1   user_id          2241 non-null   int64         
 2   viewed_at        2241 non-null   datetime64[ns]
 3   document_title   2241 non-null   object        
 4   course_name      2241 non-null   object        
 5   university_name  2241 non-null   object        
 6   category_id      2241 non-null   int64         
 7   category_name    1916 non-null   object        
 8   premium          2241 non-null   int64         
 9   rating           2241 non-null   int64         
 10  pages            2241 non-null   int64         
 11  word_count       2241 non-null   int64         
 12  id               2241 non-null   int64         
 13  name             2241 non-null   object        
 14  last_name        2241 non-null   object 

### viewed_at как object меняем на дату

In [16]:
doc_views['viewed_at'] = pd.to_datetime(doc_views['viewed_at'])

### паспортные данные в строку, был int64

In [18]:
doc_views['passport_number'] = doc_views['passport_number'].astype(str)

In [20]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3418 entries, 0 to 3417
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              3418 non-null   int64         
 1   created_at      3418 non-null   datetime64[ns]
 2   user_id         3382 non-null   float64       
 3   amount          3418 non-null   int64         
 4   brand_code      3418 non-null   object        
 5   shopper_locale  3418 non-null   object        
 6   currency        3418 non-null   object        
 7   event_code      3418 non-null   object        
 8   is_trial        3418 non-null   int64         
 9   trial_days      3418 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 267.2+ KB


### Меняем user_id тип float

In [21]:
payments['user_id'] = payments['user_id'].astype('int')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

###До тех пор пока в какой-либо графе данных присутствуют пропуски, конвертировать значение этой графы не удастся.

In [22]:
payments.query('user_id.isna()')

,id,created_at,user_id,amount,brand_code,shopper_locale,currency,event_code,is_trial,trial_days
0,2663933,2020-11-30 23:58:31,NaN,0,visa,no,NOK,AUTHORISATION,1,30
1,2663930,2020-11-30 23:57:31,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
2,2663858,2020-11-30 23:40:27,NaN,0,visa,no,NOK,AUTHORISATION,1,30
3,2663852,2020-11-30 23:39:02,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
4,2663817,2020-11-30 23:31:13,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
5,2663801,2020-11-30 23:27:48,NaN,0,visa,no,NOK,AUTHORISATION,1,30
6,2663770,2020-11-30 23:18:39,NaN,0,paypal,no,NOK,AUTHORISATION,1,30
7,2663763,2020-11-30 23:16:56,NaN,0,visa,no,NOK,AUTHORISATION,1,30
8,2663762,2020-11-30 23:16:19,NaN,0,visa,en_us,NOK,AUTHORISATION,1,30
9,2663743,2020-11-30 23:08:27,NaN,0,visa,no,NOK,AUTHORISATION,1,30


###  Удаляем строки с пустым user_id

In [23]:
payments = payments.dropna(subset=['user_id'])

### Меняем тип user_id c float на int

In [24]:
payments['user_id'] = payments['user_id'].astype(int)

In [25]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3382 entries, 36 to 3417
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              3382 non-null   int64         
 1   created_at      3382 non-null   datetime64[ns]
 2   user_id         3382 non-null   int64         
 3   amount          3382 non-null   int64         
 4   brand_code      3382 non-null   object        
 5   shopper_locale  3382 non-null   object        
 6   currency        3382 non-null   object        
 7   event_code      3382 non-null   object        
 8   is_trial        3382 non-null   int64         
 9   trial_days      3382 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 290.6+ KB


### Проверяем пропуски в doc_views

In [26]:
doc_views.isna().sum()

document_id          0
user_id              0
viewed_at            0
document_title       0
course_name          0
university_name      0
category_id          0
category_name      325
premium              0
rating               0
pages                0
word_count           0
id                   0
name                 0
last_name            0
passport_number      0
dtype: int64

In [27]:
doc_views.query('category_name.isna()')

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
20,2057305,11601338,2020-11-30 22:57:52,Sammendrag Bedriften HIS3410,Bedriften,Handelshøyskolen BI,7,NaN,1,28,43,11852,21,Catherine,Moore,5913467767
21,2695140,13049904,2020-11-30 22:57:50,Eksamen 2017,Personalpsykologi,Universitetet i Bergen,4,NaN,1,11,7,4241,22,Arthur,Smith,2752158462
22,2548470,11588852,2020-11-30 22:57:44,Bedriften,Bedriften,Handelshøyskolen BI,3,NaN,1,22,28,4448,23,Ashley,Turner,6746832890
23,3622849,13205250,2020-11-30 22:57:36,STV1400 – OPA – Samleark 1,Offentlig politikk og administrasjon 1 (innfør...,Universitetet i Oslo,7,NaN,1,7,47,6089,24,Mark,Warren,1886590146
24,6092770,11588852,2020-11-30 22:57:17,Web Page - arbeidskrav 5,Bedriftsøkonomisk analyse,Handelshøyskolen BI,8,NaN,1,5,3,487,25,Edward,Benson,9935724825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,4039156,12379916,2020-11-30 22:13:52,Organisasjonsstruktur,Organisasjonsteori og ledelse,Norges Arktiske Universitet,7,NaN,0,7,5,494,341,Lauren,Washington,6327328228
341,3910049,12820726,2020-11-30 22:13:36,"Eksamen 12 Desember 2018, svar",Etikk og samfunnsansvar,Universitetet i Sørøst-Norge,4,NaN,0,9,6,4785,342,Melissa,Morgan,1111171306
342,9161634,9620122,2020-11-30 22:13:35,"Eksamen 5 Mai 2020, svar",Sosiale medier,Handelshøyskolen BI,4,NaN,1,0,13,4740,343,Tammy,Bennett,7369891115
343,2783840,12972078,2020-11-30 22:13:13,"Eksamen 2015, svar",Velferd og økonomisk politikk,Universitetet i Bergen,4,NaN,1,0,2,442,344,Adam,Hood,7535548383


### мы прописываем команду, которая отсеет те строки, в которых нет пропусков, а в квадратных скобках передадим только те поля, которые нас интересуют

In [28]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']]

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
2,3,Lecture notes
3,8,Mandatory assignments
4,7,Summaries
...,...,...
2236,4,Past exams
2237,6,Practical
2238,4,Past exams
2239,7,Summaries


### Редактируем команду выше, добавляем удаление дублей

In [29]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates()

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
3,8,Mandatory assignments
6,4,Past exams
13,10,Essays
351,5,Other
366,9,Tutorial work
370,6,Practical
1439,1,Book solutions


### Порядок строк сбивается

In [30]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True)

,category_id,category_name
0,3,Lecture notes
1,7,Summaries
2,8,Mandatory assignments
3,4,Past exams
4,10,Essays
5,5,Other
6,9,Tutorial work
7,6,Practical
8,1,Book solutions


## Создаем словарь

### мы установим индекс как category_id и уберём крайнюю левую колонку.

In [34]:
doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True).set_index('category_id')['category_name']

category_id
3             Lecture notes
7                 Summaries
8     Mandatory assignments
4                Past exams
10                   Essays
5                     Other
9             Tutorial work
6                 Practical
1            Book solutions
Name: category_name, dtype: object

### Присваиваем название словарю category_dict:

In [35]:
category_dict = doc_views.query('category_name.isna() == False')[['category_id', 'category_name']].drop_duplicates().reset_index(drop=True).set_index('category_id')['category_name']

In [36]:
 category_dict[10]

'Essays'

## нужно пересобрать датасет, чтобы избавиться от пропусков и ориентироваться только на category_id, где нет пропусков, и по category_id заполнить графу category_name, вставив значения вместо имеющихся пропусков.

In [37]:
def fill_nans(col_id): return category_dict[col_id]

In [38]:
doc_views['category_name'] = doc_views['category_id'].apply(fill_nans)

In [39]:
 doc_views.isna().sum()

document_id        0
user_id            0
viewed_at          0
document_title     0
course_name        0
university_name    0
category_id        0
category_name      0
premium            0
rating             0
pages              0
word_count         0
id                 0
name               0
last_name          0
passport_number    0
dtype: int64

# Анонимизация данных. Обработка дублей

## Анонимизация данных

In [40]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,Edwards,1903155229
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,Smith,1466748063
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,Parker,4864835140
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,Mcguire,1837181476
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,Erickson,5567324499


### функции hash_passport и apply

In [41]:
def hash_passport(passport): return hashlib.sha256(passport.encode()).hexdigest()

In [42]:
doc_views['passport_number'] = doc_views['passport_number'].apply(hash_passport)

In [43]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,last_name,passport_number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,Edwards,7d41dbb1caed125dc35cf0018f298844c8d1c74ffb00e2...
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,Smith,3032fb8eb98de582cd11a0eeb1f94c327aea2413dfcdd5...
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,Parker,5c1079836d9b7ddaeabcc67d031a5a7b2bb5c22173977c...
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,Mcguire,46d77f6d89836f8864188ebd124882789c214933820ece...
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,Erickson,6a5f121ac346a55ce0f888a297effc6cfa48aeceddcd33...


### Удалим фамилии

In [44]:
doc_views = doc_views.drop(columns='last_name')

In [45]:
doc_views.head()

,document_id,user_id,viewed_at,document_title,course_name,university_name,category_id,category_name,premium,rating,pages,word_count,id,name,passport_number
0,9738742,8469118,2020-11-30 22:59:56,Sosiale medier - 2. time,Sosiale medier,Handelshøyskolen BI,3,Lecture notes,0,0,1,151,1,Jacob,7d41dbb1caed125dc35cf0018f298844c8d1c74ffb00e2...
1,2042372,13198098,2020-11-30 22:59:52,Bedriften-kortsvarsspørsmål,Bedriften,Handelshøyskolen BI,7,Summaries,1,50,23,9551,2,Felicia,3032fb8eb98de582cd11a0eeb1f94c327aea2413dfcdd5...
2,3653898,11182794,2020-11-30 22:59:41,Sammendrag-i-markedsføring,Markedsføring-grunnkurs,Norges teknisk-naturvitenskapelige universitet,3,Lecture notes,0,1,107,37572,3,Erin,5c1079836d9b7ddaeabcc67d031a5a7b2bb5c22173977c...
3,7968789,13004670,2020-11-30 22:59:37,"Dydsetikk og abort, Hursthouse",Examen philosophicum,Universitetet i Oslo,8,Mandatory assignments,1,3,6,929,4,Morgan,46d77f6d89836f8864188ebd124882789c214933820ece...
4,2036442,12525936,2020-11-30 22:59:30,Kortsvar-bedriften - Sammendrag Bedriften,Bedriften,Handelshøyskolen BI,7,Summaries,1,228,21,5567,5,Lisa,6a5f121ac346a55ce0f888a297effc6cfa48aeceddcd33...


## Обработка дублей

In [46]:
doc_views.duplicated().sum()

np.int64(0)

In [47]:
payments.duplicated().sum()

np.int64(48)

### Удаляем 48 дублей

In [48]:
payments = payments.drop_duplicates()